In [69]:
%matplotlib inline
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

import numpy as np
import scipy.misc
import word2vec
import scipy.signal
import time
import os
import pickle
# If we want proper CUDA debug info.
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from gensim.scripts.glove2word2vec import glove2word2vec

import torch
from torch.autograd import Variable
from gensim.models.keyedvectors import KeyedVectors

import torch.nn.functional as F

from matplotlib import rcParams
rcParams['axes.grid'] = False


In [71]:
glove_dir = "/Users/jawa/Desktop/mark_ra/falconet/models/resources/"
glove_dim = 100
glove_file = "glove.twitter.27B.{}d.txt".format(glove_dim)
glove2word2vec(glove_input_file=glove_dir+glove_file, word2vec_output_file="resources/gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("resources/gensim_glove_vectors.txt", binary=False)


In [104]:
glove_model['idk']

array([ 2.3352e-01,  9.3655e-02, -1.4955e-01,  4.5389e-01, -2.9652e-01,
        3.9855e-01,  5.8720e-01,  3.7310e-01,  4.0705e-01,  1.2912e+00,
       -2.5254e-01, -3.3421e-01, -4.0356e+00,  5.4054e-01,  3.7914e-01,
        6.5204e-01,  2.0346e-01, -3.6562e-01, -6.0288e-01, -5.6573e-01,
        4.8248e-01,  6.5377e-02, -2.6782e-01,  8.0191e-02,  1.9383e-01,
       -3.8029e-01, -3.9171e-01, -3.4809e-01, -7.1972e-01, -2.3332e-01,
       -1.6676e-01,  2.1976e-01,  3.2141e-02, -3.3264e-01, -1.0799e+00,
       -2.2029e-01,  2.2205e-01, -1.5094e-01,  2.9377e-01, -1.3618e-01,
       -1.0509e+00, -1.0690e-01,  3.6625e-01, -1.4283e-01, -5.7559e-01,
       -9.2575e-02, -7.0247e-02, -1.8954e-01, -4.8209e-01,  1.3509e-01,
       -4.7616e-01,  2.8590e-01, -4.7091e-01, -2.4828e-01,  5.6715e-01,
       -1.9084e-01, -4.1764e-02,  4.5559e-02,  6.8688e-01,  3.4829e-02,
        2.9223e-03, -6.9852e-02,  6.0892e-01, -6.0985e-01,  2.4361e-01,
       -2.0014e-01, -2.2803e-01,  2.9203e-01, -1.2023e-01, -7.34

In [98]:
glove_model['king'] - glove_model['queen']

array([ 0.71940005, -0.16053   , -0.02105001, -0.400193  ,  0.01777002,
        0.174238  , -0.27057698, -0.09550999, -0.01642901,  0.38466   ,
        0.52881   , -0.07535   , -0.47440004,  0.33339006,  0.28133997,
       -0.36134   , -0.05463001,  0.12857   ,  0.66929996, -0.20911801,
       -0.13664001, -0.13915   , -0.184277  , -0.72419   ,  0.17056006,
        0.02060997, -0.22682999, -0.518969  , -0.31137002, -0.47429   ,
       -0.10251001,  0.48950002, -0.339654  , -0.417704  ,  0.404462  ,
        0.274409  , -0.19733599, -0.18448   ,  0.26866   , -0.45244998,
        0.06341399,  0.35621   , -0.58125997, -0.232124  ,  0.49095   ,
       -0.17906001, -0.375117  ,  0.44752002,  0.660368  ,  0.243958  ,
       -0.32624   , -0.32801798,  0.160478  ,  0.76563   , -0.79741   ,
        0.03702003,  0.403962  , -0.6136    ,  0.573618  , -0.45720997,
       -0.01168001, -0.69703996,  0.1785654 , -0.304301  ,  0.11383002,
       -0.96731395,  0.58383   , -0.28404   , -0.76435   , -0.44

In [26]:
# pre process model_dir
model_dir = "data/processed/nans,2000_maxlength,26_minwcount,0_nlp,mcb_pad,left_trainsplit,train"
# aid_to_ans.pickle
aid_to_ans = pickle.load(open(os.path.join(model_dir,"aid_to_ans.pickle"),"rb"))
# ans_to_aid.pickle
ans_to_aid = pickle.load(open(os.path.join(model_dir,"ans_to_aid.pickle"),"rb"))
# testdevset.pickle
testdevset = pickle.load(open(os.path.join(model_dir,"testdevset.pickle"),"rb"))
# testset.pickle
testset = pickle.load(open(os.path.join(model_dir,"testset.pickle"),"rb"))
# trainset.pickle
trainset = pickle.load(open(os.path.join(model_dir,"trainset.pickle"),"rb"))
# valset.pickle
valset = pickle.load(open(os.path.join(model_dir,"valset.pickle"),"rb"))
# wid_to_word.pickle
wid_to_word = pickle.load(open(os.path.join(model_dir,"wid_to_word.pickle"),"rb"))
# word_to_wid.pickle
word_to_wid = pickle.load(open(os.path.join(model_dir,"word_to_wid.pickle"),"rb"))

In [118]:
trainset[10]

{'answer': 'yes',
 'answer_aid': 0,
 'answers': ['yes'],
 'answers_aid': [0],
 'answers_count': [10],
 'answers_occurence': [['yes', 10]],
 'image_name': 'COCO_train2014_000000393221.jpg',
 'question': 'Is the sky blue?',
 'question_id': 393221000,
 'question_length': 4,
 'question_wids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  12,
  33,
  34],
 'question_words': ['is', 'the', 'sky', 'blue'],
 'question_words_UNK': ['is', 'the', 'sky', 'blue'],
 'seq_length': 4}

In [119]:
class VisualQuestionsDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, ds,ans_to_aid,aid_to_ans,wid_to_word,word_to_wid,image_root_dir=None,transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.question_answer_ds =  ds
        self.ans_to_aid = ans_to_aid
        self.aid_to_ans = aid_to_ans
        self.wid_to_word = wid_to_word
        self.word_to_wid = word_to_wid
        self.image_root_dir = image_root_dir

    def __len__(self):
        return len(self.question_answer_ds)

    def __getitem__(self, idx):
        
        # self.image_root_dir
        image_feat = None
        
        question = self.question_answer_ds[idx]['question_wids']
        empty_vec = np
        question_vec = [glove_model[wid_to_word[x]] for x in question if x!=0 ]
        answer_id = self.question_answer_ds[idx]['answer_aid']
        return question_vec,answer_id,image_feat

In [120]:
test_dataset = VisualQuestionsDataset(trainset,ans_to_aid,aid_to_ans,wid_to_word,word_to_wid)

for i in range(len(test_dataset)):
    question,answer,_ = test_dataset[i]
    print(len(question))
    print(answer)
    break




7
934
